In [1]:
from pathlib import Path
import numpy as np
import dask.array as da
import mmap
import pandas as pd
import scipy
from scipy import interpolate
from scipy import fftpack
from scipy import signal
import matplotlib.pyplot as plt


def other_mmap(filename, shape, dtype, chunks):
    with open(filename, "rb") as fh:
        # `mmap` duplicates the file descriptor
        # `0` means map the full file
        mm = mmap.mmap(fh.fileno(), 0, access=mmap.ACCESS_READ)

    # coerce to NumPy array of expected type and shape
    a = np.asarray(mm).view(dtype).reshape(shape)

    # Don't call `asarray`
    # name=False to avoid hashing
    a_d = da.from_array(a, chunks=chunks, asarray=False, name=False)
    return a_d

folder_base = Path('C:/Users/AudreyHay/Desktop/openephys/2024-02-16_11-11-21/Record Node 102/experiment1/recording2/continuous/Acquisition_Board-104.Rhythm Data')
#folder_base = Path('/mnt/data/ahay/')

filename = folder_base / f'continuous.dat'
data = other_mmap(filename, (-1, 64), "int16", chunks=(-1, 1))
data2 = data[:,1:16] #33:48 pour le 2e rat
data3 = data[:,16:32] #48:64 pour le 2e rat

list_rawchan = np.append(data2, data3, axis = 1)
list_rawchan2 = list_rawchan.T
list_rawchan_filt = []
for ele, file in enumerate(list_rawchan2):
    ele2 = ele + 1
    sampling_rate = 2000 # Hz
    new_sampling_rate = 1000 # Hz
    ratio = int(sampling_rate/new_sampling_rate)
    nmberchann = file.shape
    datalen = file.size/ nmberchann[0]# put 1 when no np.append for list_rawchan
    Nmber_points = int(datalen * new_sampling_rate / sampling_rate)
    globals()[f"DS_GoodChannels_{ele2}"] = signal.decimate(file, ratio, ftype='fir', axis= 0, zero_phase=True)
    Good_channels_filtered = globals()[f"DS_GoodChannels_{ele2}"]
    #Good_channels_filtered = Good_channels_filtered[:,1:32]
    list_rawchan_filt.append(Good_channels_filtered)
    globals()[f"xrGoodChannels_{ele2}"] = None

list_rawchan = None
folder_output = Path('C:/Users/AudreyHay/Desktop/openephys/2024-02-16_11-11-21/Record Node 102/experiment1/recording2/continuous/Acquisition_Board-104.Rhythm Data')
filename2 = folder_output / f'RawDataChannelExtractedDS2.npy'
np.save(filename2, list_rawchan_filt)

c:\Users\AudreyHay\anaconda3\envs\env\lib\site-packages\dask\array\core.py:1326: FutureWarning: The `numpy.append` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  warnings.warn(
